In [1]:
#reading in file
import glob
import numpy as np
#import pandas as pd
import sys
sys.path.insert(1, './')
import functions

filename = sorted(glob.glob("p0.0_n20_rcut2.5_T0.50_run1/run_??[1-2].dump"))

data, nlin, nlog, l_box, time = functions.read_lammps(filename)
data = np.reshape(data, (nlin,nlog, data.shape[1], data.shape[2]))
#removing the first conf from each loop
data = data[:,1:,:,:]
nlog = nlog -1
print(data.shape, nlin, nlog)

# ordeiing the ids of the particles
for i in range(nlin):
    for j in range(nlog):
        data[i,j,:,:] = data[i,j,data[i,j,:,0].astype(int).argsort(),:]



id_a, type_a, mol_a, pos_a, vel_a = data[:,:,:,0].astype(int), data[:,:,:,1].astype(int), data[:,:,:,2].astype(int), data[:,:,:,3:6].astype(float), data[:,:,:,6:9].astype(float)  
pos_a = pos_a.astype(float)
l_box = l_box.astype(float)
#data_orig = data.copy()
pos_a_orig = pos_a.copy()
mol_a_orig = mol_a.copy()

(10, 21, 8000, 10) 10 21


In [ ]:
# remove the COM drift
pos_a = pos_a_orig.copy()
mol_a = mol_a_orig.copy()
pos_a = np.reshape(pos_a, (nlin*nlog, pos_a.shape[2], pos_a.shape[3]))
pos_a = functions.removing_com_drift(pos_a,l_box[0,:])
pos_a = np.reshape(pos_a, (nlin, nlog, pos_a.shape[1], pos_a.shape[2]))

pos_a = np.reshape(pos_a, (nlin*nlog, pos_a.shape[2], pos_a.shape[3]))
mol_a = np.reshape(mol_a, (nlin*nlog, mol_a.shape[2]))
#calculating the gyration tensor and rg
gyration = functions.gTensor(pos_a, l_box[0,:], mol_a)
gyration = np.reshape(gyration, (gyration.shape[0]*gyration.shape[1], gyration.shape[2]))
np.save("gyration_tensor",gyration)
gyration = np.load("gyration_tensor.npy")
rg_val, eigen_val, eigen_vec = functions.rg(gyration)
np.save("rg",rg_val)
np.save("eigenval",eigen_val)
np.save("eigenvec",eigen_vec)



In [3]:
print(rg_val.shape)

AttributeError: 'tuple' object has no attribute 'shape'